<a href="https://colab.research.google.com/github/isaacmsl/mnist-classifiers-california-housing-regressors/blob/main/california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import tree, metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np

In [ ]:

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(tree.DecisionTreeRegressor(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=1)


print(grid_search.fit(X_train, y_train))
print(grid_search.best_params_)

print(grid_search.best_score_)

regressor = grid_search.best_estimator_


regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


print(metrics.mean_squared_error(y_test, y_pred))
print(metrics.r2_score(y_test, y_pred))

Fitting 3 folds for each of 36 candidates, totalling 108 fits
GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             verbose=1)
{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
0.6893996163202178
0.40832623099687676
0.688397868316641
